<table  align="left" width="100%"> <tr>
        <td  style="background-color:#ffffff;"><a href="https://qsoftware.lu.lv/index.php/qworld/" target="_blank"><img src="..\images\qworld.jpg" width="35%" align="left"></a></td>
        <td  align="right" style="background-color:#ffffff;vertical-align:bottom;horizontal-align:right">
            prepared by Caner ERCAN (<a href="http://qworld.lu.lv/index.php/qturkey/" target="_blank">QTurkey</a>)
        </td>        
</tr></table>

<table width="100%"><tr><td style="color:#bbbbbb;background-color:#ffffff;font-size:11px;font-style:italic;text-align:right;">This cell contains some macros. If there is a problem with displaying mathematical formulas, please run this cell to load these macros. </td></tr></table>
$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$ \newcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \newcommand{\dot}[2]{ #1 \cdot #2} $
$ \newcommand{\biginner}[2]{\left\langle #1,#2\right\rangle} $
$ \newcommand{\mymatrix}[2]{\left( \begin{array}{#1} #2\end{array} \right)} $
$ \newcommand{\myvector}[1]{\mymatrix{c}{#1}} $
$ \newcommand{\myrvector}[1]{\mymatrix{r}{#1}} $
$ \newcommand{\mypar}[1]{\left( #1 \right)} $
$ \newcommand{\mybigpar}[1]{ \Big( #1 \Big)} $
$ \newcommand{\sqrttwo}{\frac{1}{\sqrt{2}}} $
$ \newcommand{\dsqrttwo}{\dfrac{1}{\sqrt{2}}} $
$ \newcommand{\onehalf}{\frac{1}{2}} $
$ \newcommand{\donehalf}{\dfrac{1}{2}} $
$ \newcommand{\hadamard}{ \mymatrix{rr}{ \sqrttwo & \sqrttwo \\ \sqrttwo & -\sqrttwo }} $
$ \newcommand{\vzero}{\myvector{1\\0}} $
$ \newcommand{\vone}{\myvector{0\\1}} $
$ \newcommand{\stateplus}{\myvector{ \sqrttwo \\  \sqrttwo } } $
$ \newcommand{\stateminus}{ \myrvector{ \sqrttwo \\ -\sqrttwo } } $
$ \newcommand{\myarray}[2]{ \begin{array}{#1}#2\end{array}} $
$ \newcommand{\X}{ \mymatrix{cc}{0 & 1 \\ 1 & 0}  } $
$ \newcommand{\Z}{ \mymatrix{rr}{1 & 0 \\ 0 & -1}  } $
$ \newcommand{\Htwo}{ \mymatrix{rrrr}{ \frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} } } $
$ \newcommand{\CNOT}{ \mymatrix{cccc}{1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0} } $
$ \newcommand{\norm}[1]{ \left\lVert #1 \right\rVert } $
$ \newcommand{\pstate}[1]{ \lceil \mspace{-1mu} #1 \mspace{-1.5mu} \rfloor } $

<h1> Original QPA </h1>

In this notebook, we will learn how to realize the algorithm in ququarts, by qiskit. Reason of calling "original" is that we can also realize optimal version of the algorithm that has less quantum gates. Therefore, that version has more success probability when you want to run on a real quantum processor. But original version first...

Consider a linear transformation such $\ket{\phi_{m}^\pm}= P_{m}^\pm\ket{\phi}$ with $m = 0,1,...,d-1$ in d dimension discrete space spanned by orthonormal set of vectors $S=\{\ket{0},\ket{1},...,\ket{d-1}\}$.  The operator $P_{m}^\pm$ are $dxd$ matrices and they permute the components of a given vector $\ket{\phi}$ in a cyclic manner as 

$$
P_m^{\pm}= \sum_{k=0}^{d-1} \ket{(m{\pm}k)_{\mod(4)}}\bra{k} ~~~~~(1)
$$

where + and - indicates the parity of the permutation. In fact, these cyclic permutations are a subset of all possible permutations. 

> Remember, the permutation problem asks that if we are given an arbitrary black box how many queries are needed to estimate the parity of the corresponding permutation? 

<h2> Classical Approach</h2>

As shown in the following figure, the vector components can be interpreted as different "wires" entering one of permutation gates(or black box) since they correspond to different physical states. It can be seen that a single query is not enough to determine the parity of the permutation applied when we start with the input state $\ket{1}$. The figure demonstrates that the input $\ket{01}$ gives the same output $\ket{11}$ for two different permutation operations $P_2^{+}$ and $P_0^{-}$. In fact, the same case is valid for any other input states. The components of a vector correspond to the probability amplitudes which are represented by a single wire in the circuit diagram as shown in the following figure. 

<img src="../images/qubit_as_wire.png" width="70%" align="middle">

<h3>Task 1</h3>

Please, show that the problem can not be solved for these two gates and draw the circuit: 

In [ ]:
# import all necessary objects and methods for quantum circuits
from qiskit import *
from numpy import pi

# define a quantum register with a single qubit
q = QuantumRegister(2,"q")
# define a classical register with a single bit
c = ClassicalRegister(2,"c")
# define a quantum circuit
qc = QuantumCircuit(q,c)



qc.measure(q,c)
job = execute(qc,Aer.get_backend('qasm_simulator'),shots=1000)
counts = job.result().get_counts(qc)   
print(counts)

qc.draw("mpl")

<a href="F02_Original_QPA_Solutions.ipynb#task1">click for our solution</a>

<h2> Quantum Approach</h2>

Here, d is the dimension of quantum states and j is input state as a decimal i.g. the number 3 in decimal equals 11 in binary. After starting with the initial state $\ket{j}$, the permutation operator $P_m^{\pm}$ is sandwiched between Quantum Fourier Transform $U_{FT}$ and its inverse $U^\dagger_{FT}$. The measurement over the state $\ket{\psi_{3,m}^\pm}$ results in either $\ket{j}$ or $\ket{(d-j)_{\mod(d)}}$ with unit probability if the parity of the permutation is positive or negative, respectively. 

<img src="../images/circuit_for_ququarts.png" width="80%" align="middle">

Let's look into this circuit gate by gate:
We start the input state $\ket{j}$ that will be probably $\ket{1}$ later. The quantum algorithm starts to solve the problem by initializing the following state

$$
QFT\ket{j} = \ket{\psi_1} = \frac{1}{\sqrt{d}}\sum_{k=0}^{d-1} e^{i\frac{2\pi j k }{d}}\ket{k} ~~~~~~(2)
$$

And then, one of the permutation operators in the equation (1) is applied 

$$
\ket{\psi_{2,m}^\pm} = P_m^{\pm}QFT\ket{j} = \frac{1}{\sqrt{d}}\sum_{k=0}^{d-1} e^{i\frac{2\pi j k }{d}}\ket{(m{\pm}k)_{\mod(4)}} ~~~~~(3)
$$

Lastly, an inverse QFT is applied to remove superposition into one of either 

$$
\ket{\psi_{3,m}^+} = QFT^{\dagger}P_m^{+}QFT\ket{j} = e^{i\frac{2\pi m j }{d}}\ket{j}~~~~~(4)
$$

or

$$
\ket{\psi_{3,m}^-} = QFT^{\dagger}P_m^{-}QFT\ket{j} = e^{-i\frac{2\pi (d-j)m}{d}}\ket{(d-j)_{\mod(d)}}~~~~~(5)
$$

if the parity of the given permutation is positive or negative, respectively. Therefore, a single query is adequate to solve the problem deterministically after measuring $\ket{\psi_{3,m}^-}$ in the $S$ basis. The phases uniquely assigned beaacuse the components of $\ket{\psi_{1}}$ play a key role in this computational speed-up by keeping track of the parity, which is classically impossible.

<h2>Summation Method</h2>

Let's do some example in the following to convince ourselves:

$QFT^{\dagger} P^{+}_2 QFT \ket{01}=?$

To find answer of this question, the equations given above can be used. For now, I'm beginning to find the $\ket{\psi_2}$ by using the equation (3):

$\ket{\psi_2^+} 
= P_2^+QFT\ket{01} 
= P_2^+QFT\ket{1} 
= \frac{1}{\sqrt{4}}\sum_{k=0}^{4-1} e^{i\frac{2\pi 1 k } {4}}\ket{(2{+}k)_{\mod(4)}} \\
= \frac{1}{2}\sum_{k=0}^{3} e^{i\frac{\pi k }{2}}\ket{(2{+}k)_{\mod(4)}} 
= \frac{1}{2}(e^{i\frac{\pi 0 }{2}}\ket{(1{+}0)_{\mod(4)}}+ e^{i\frac{\pi 1 }{2}}\ket{(2{+}1)_{\mod(4)}} + e^{i\frac{\pi 2 }{2}}\ket{(2{+}2)_{\mod(4)}} + e^{i\frac{\pi 3 }{2}}\ket{(2{+}3)_{\mod(4)}}) \\
= \frac{1}{2}(\ket{2} + e^{i\frac{\pi }{2}}\ket{3} + e^{i\frac{\pi 2 }{2}}\ket{0} + e^{i\frac{\pi 3 }{2}}\ket{1} )$

and let's continue by using the equation (4) that is for positive cyclic permutations:

$\ket{\psi_3^+}
= QFT^{\dagger} P^{+}_2 QFT \ket{01} 
= \frac{1}{2}(QFT^{\dagger}\ket{2} + e^{i\frac{\pi}{2}}QFT^{\dagger}\ket{3} + e^{i\frac{\pi 2 }{2}}QFT^{\dagger}\ket{0} + e^{\frac{i\pi 3 }{2}}QFT^{\dagger}\ket{1} ) \\
= \frac{1}{2}(\frac{1}{2}\sum_{k=0}^{3} e^{i\frac{2\pi 2 k }{4}}\ket{k}+\frac{1}{2}e^{i\frac{\pi }{2}}\sum_{k=0}^{3} e^{i\frac{2\pi 3 k }{4}}\ket{k}+\frac{1}{2}e^{i\frac{\pi 2 }{2}}\sum_{k=0}^{3} e^{i\frac{2\pi 0 k }{4}}\ket{k}+\frac{1}{2}e^{i\frac{\pi 3 }{2}}\sum_{k=0}^{3}e^{i\frac{2\pi 1 k }{4}}\ket{k}) \\
= \frac{1}{4}[\ket{0} + e^{-i \pi}\ket{1} + e^{-i 2\pi}\ket{2} + e^{-i 3\pi}\ket{3}
+ e^{i\frac{2\pi}{4}}(\ket{0} + e^{-i\frac{2\pi 3}{4}}\ket{1} + e^{-i\frac{2\pi 2 3}{4}}\ket{2} + e^{-i\frac{2\pi 3 3}{4}}\ket{3})
+ e^{i\pi}(\ket{0} + \ket{1} + \ket{2} + \ket{3})
+ e^{i\frac{3\pi}{2}}(\ket{0} + e^{-i\frac{2\pi}{4}}\ket{1} + e^{-i\frac{2\pi 2}{4}}\ket{2} + e^{-i\frac{2\pi 3}{4}}\ket{3})] \\
= \frac{1}{4}[\ket{0} + e^{-i\pi}\ket{1} + e^{-i 2\pi}\ket{2} + e^{-i 3\pi}\ket{3}
+ e^{i\frac{2\pi}{4}}\ket{0} + e^{-i 3\pi}\ket{1} + e^{-i\frac{5\pi}{2}}\ket{2} + e^{-i 4\pi}\ket{3}
+ e^{i \pi}\ket{0} + e^{i \pi}\ket{1} + e^{i \pi}\ket{2} + e^{i \pi}\ket{3}
+ e^{i \frac{\pi 3}{2}}\ket{0} + e^{i \pi}\ket{1} + e^{i \frac{\pi}{2}}\ket{2} + \ket{3}] \\
= \frac{1}{4}[\ket{0} - \ket{1} + \ket{2} -\ket{3} + i\ket{0} -\ket{1} -i\ket{2} + \ket{3} -\ket{0} -\ket{1} -\ket{2} -\ket{3} -i\ket{0} -\ket{1} + i\ket{2} + \ket{3}] \\
= \frac{1}{4}[-4\ket{1}] = -\ket{01}$

If you look at the equation (4), you will see the equality such below: 
$\ket{\psi_3^+} = e^{i\frac{2\pi j m}{d}}\ket{j} = e^{i\frac{2\pi 1 2}{4}}\ket{1} = e^{i\pi}\ket{1} \equiv -\ket{01}$

Namely, we can see that the results and formulas are consistent with each other

<h3>Task 2</h3>

Please, implement the following iteration by using summation method: 

$QFT^{\dagger} P^{-}_0 QFT \ket{01}=?$

<a href="F02_Original_QPA_Solutions.ipynb#task2">click for our solution</a>

<h2>Matrix Method</h2>

Matrix method actually is the serial version summation method. Namely, we can solve these questions by using matrices, that is faster than the other one. You can use conventional quantum Fourier transform matrix now. 

$$ Q_{FT} = \frac{1}{2}\mymatrix{rrrr}{1 & 1 & 1 & 1\\ 
                   1 & i & -1 & -i \\ 
                   1 & -1 & 1 & -1 \\
                   1 & -i & -1 & i} \mbox{ , }  Q_{FT}^\dagger =\frac{1}{2}\mymatrix{rrrr}{1 & 1 & 1 & 1\\ 
                   1 & -i & -1 & i \\ 
                   1 & -1 & 1 & -1 \\
                   1 & i & -1 & -i}$$

Let's do some example in the following to convince ourself:

$QFT^{\dagger} P^{-}_0 QFT \ket{11}=?$

$=\frac{1}{2}\mymatrix{rrrr}{1 & 1 & 1 & 1\\ 
                   1 & -i & -1 & i \\ 
                   1 & -1 & 1 & -1 \\
                   1 & i & -1 & -i} \mymatrix{rrrr}{0 & 1 & 0 & 0\\ 
                   0 & 0 & 1 & 0 \\ 
                   0 & 0 & 0 & 1 \\
                   1 & 0 & 0 & 0} \frac{1}{2}\mymatrix{rrrr}{1 & 1 & 1 & 1\\ 
                   1 & i & -1 & -i \\ 
                   1 & -1 & 1 & -1 \\
                   1 & -i & -1 & i}\myvector{0 \\ 0 \\ 0 \\ 1} = \frac{1}{4}\mymatrix{rrrr}{1 & 1 & 1 & 1\\ 
                   1 & -i & -1 & i \\ 
                   1 & -1 & 1 & -1 \\
                   1 & i & -1 & -i} \mymatrix{rrrr}{0 & 1 & 0 & 0\\ 
                   0 & 0 & 1 & 0 \\ 
                   0 & 0 & 0 & 1 \\
                   1 & 0 & 0 & 0}\myvector{1 \\ -i  \\ -1 \\ i}
= \frac{1}{4}\mymatrix{rrrr}{1 & 1 & 1 & 1\\ 
                   1 & -i & -1 & i \\ 
                   1 & -1 & 1 & -1 \\
                   1 & i & -1 & -i}\myvector{-i  \\ -1 \\ i \\ 1}=\frac{1}{4}\myvector{-i -1 +i +1 \\ -i +i -i +i \\-i +1 -1 +i   \\ -i -i -i -i} = -i\myvector{0 \\ 0 \\ 0 \\ 1} =-i\ket{11}$

<h3>Task 3</h3>

Please, implement the following iteration by using matrix method and compare with the result from the summation method: 

$QFT^{\dagger} P^{+}_2 QFT \ket{01}=?$

<a href="F02_Original_QPA_Solutions.ipynb#task3">click for our solution</a>

> Because there are $4$ different input and $8$ different permutation for four dimension, totally, you can do $32$ different iteration by using these two methods.

You can see the whole iterations and their results in the following table. Don't care that the permutation gates are not sandwiched fourier transforms: 

| $$\ket{00}$$ | $$\ket{01}$$ | $$\ket{10}$$ | $$\ket{11}$$ |
| :-: | :-: | :-: |  :-: | 
| $P^{+}_0\ket{00}=\ket{00}$ | $P^{+}_0\ket{01}=\ket{01}$  | $P^{+}_0\ket{10}= \ket{10}$ | $P^{+}_0\ket{11}=\ket{11}$ |
| $P^{+}_1\ket{00}=\ket{00}$ | $P^{+}_1\ket{01}=-i\ket{01}$  | $P^{+}_1\ket{10}=-\ket{10}$ | $P^{+}_1\ket{11}=i\ket{11}$ |
| $P^{+}_2\ket{00}=\ket{00}$ | $P^{+}_2\ket{01}=-\ket{01}$  | $P^{+}_2\ket{10}= \ket{10}$ | $P^{+}_2\ket{11}=-\ket{11}$ |
| $P^{+}_3\ket{00}=\ket{00}$ | $P^{+}_3\ket{01}=i\ket{01}$  | $P^{+}_3\ket{10}=-\ket{10}$ | $P^{+}_3\ket{11}=-i\ket{11}$ |
| $P^{-}_3\ket{00}=\ket{00}$ | $P^{-}_3\ket{01}=-i\ket{11}$  | $P^{-}_3\ket{10}=-\ket{10}$ | $P^{-}_3\ket{11}=i\ket{01}$ |
| $P^{-}_2\ket{00}=\ket{00}$ | $P^{-}_2\ket{01}=-\ket{11}$  | $P^{-}_2\ket{10}= \ket{10}$ | $P^{-}_2\ket{11}=-\ket{01}$ |
| $P^{-}_1\ket{00}=\ket{00}$ | $P^{-}_1\ket{01}=i\ket{11}$  | $P^{-}_1\ket{10}=-\ket{10}$ | $P^{-}_1\ket{11}=-i\ket{01}$ |
| $P^{-}_0\ket{00}=\ket{00}$ | $P^{-}_0\ket{01}=\ket{11}$  | $P^{-}_0\ket{10}= \ket{10}$ | $P^{-}_0\ket{11}=\ket{01}$ |

As you can see, the problem is just solved for inputs $\ket{01}$ and $\ket{11}$. Because you cannot determine the parity of the permutation gate applied by looking output for inputs $\ket{00}$ and $\ket{10}$.

<h2> Original QPA by Qiskit</h2>


Let's start to build the whole program by using qiskit. Firstly, we should initialize the qubit states. But we start with one of the superposed state that comes after applying quantum Fourier transform to one of four input states. Because this way is need for Quantum Fourier Transform in 2-qubit case. For example, if we apply QFT to the input $\ket{01}$, we have this superposed state $\frac{1}{2}(\ket{00}+i\ket{01}-\ket{10}-i\ket{11})$. Let's build up this state by using Qiskit and get the statevector.

In [4]:
# import all necessary objects and methods for quantum circuits
from qiskit import *
from numpy import pi

# define a quantum register with a single qubit
q = QuantumRegister(2,"q")
# define a classical register with a single bit
c = ClassicalRegister(2,"c")
# define a quantum circuit
qc = QuantumCircuit(q,c)

# apply the necessary gates to build up superposed states.
qc.h(q[0])
qc.h(q[1])
qc.s(q[0])
qc.z(q[1])

state = execute(qc,Aer.get_backend('statevector_simulator'),shots=4096)  
print(state.result().get_statevector())

[ 0.5+0.j   0. +0.5j -0.5+0.j  -0. -0.5j]


<h3>Task 4</h3>

Please, build up other three superposed states and get the statevector to convince yourself. 

In [ ]:
# import all necessary objects and methods for quantum circuits
from qiskit import *
from numpy import pi

# define a quantum register with a single qubit
q = QuantumRegister(2,"q")
# define a classical register with a single bit
c = ClassicalRegister(2,"c")
# define a quantum circuit
qc = QuantumCircuit(q,c)

# apply the necessary gates to build up superposed states.

#
#your code is here!
#

state = execute(qc,Aer.get_backend('statevector_simulator'),shots=4096)  
print(state.result().get_statevector())

<a href="F02_Original_QPA_Solutions.ipynb#task4">click for our solution</a>

<h3>Task 5</h3>

Please, build up the whole algorithm for the input state $\ket{01}$ and apply all permutation gates. Draw the circuit and evaluate the results to convince yourself. 

In [ ]:
# import all necessary objects and methods for quantum circuits
from qiskit import *
from numpy import pi

# define a quantum register with a single qubit
q = QuantumRegister(2,"q")
# define a classical register with a single bit
c = ClassicalRegister(2,"c")
# define a quantum circuit
qc = QuantumCircuit(q,c)

qc.h(q[0])
qc.h(q[1])
qc.s(q[0])
qc.z(q[1])
#qc.z(q[0])
qc.barrier()


#
#your code is here!
#

qc.barrier()

qc.cx(q[1],q[0])
qc.h(q[1])
qc.h(q[0])
qc.cx(q[1],q[0])
qc.h(q[1])
qc.h(q[0])
qc.cx(q[1],q[0])

qc.h(q[0])
qc.cx(q[1],q[0])
qc.t(q[0])
qc.cx(q[1],q[0])
qc.tdg(q[0])
qc.tdg(q[1])
qc.h(q[1])
qc.barrier()

qc.measure(q,c)
job = execute(qc,Aer.get_backend('qasm_simulator'),shots=4096)
counts = job.result().get_counts(qc)   
print(counts)

qc.draw("mpl")

<a href="F02_Original_QPA_Solutions.ipynb#task5">click for our solution</a>